In [1]:
#!pip install spacy==2.3.5
#!python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('New Folder\zomato.csv')

In [4]:
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [5]:
# Group and aggregate duplicate restaurants that are listed under multiple types in listed_in(type)
grouped = data.groupby(["name", "address"]).agg({"listed_in(type)" : list})
newdata = pd.merge(grouped, data, on = (["name", "address"]))
#newdata.index = newdata["name"]
# Drop rows which have duplicate information in "name", "address" and "listed_in(type)_x"
newdata["listed_in(type)_x"] = newdata["listed_in(type)_x"].astype(str) # converting unhashable list to a hashable type
newdata.drop_duplicates(subset = ["name", "address", "listed_in(type)_x"], inplace = True)

In [6]:
newdata.dropna(inplace=True)
# slice to reduce the time complexity to run
newdata = newdata[:4]

In [7]:
# Transforming the target (restaurant ratings)

# Extracting the first three characters of each string in "rate"
newdata["rate"] = newdata["rate"].str[:3] 

# Removing rows with "NEW" in ratings as it is not a predictable level
newdata = newdata[newdata.rate != "NEW"] 
newdata = newdata[newdata.rate != "-"] 

# Dropping rows that have missing values in ratings 
newdata = newdata.dropna(subset = ["rate"])

In [8]:
newdata.head(2)

,name,address,listed_in(type)_x,url,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type)_y,listed_in(city)
2,#L-81 Cafe,"Sector 6, HSR Layout, HSR","['Delivery', 'Dine-out', 'Delivery', 'Dine-out...",https://www.zomato.com/bangalore/l-81-cafe-hsr...,Yes,No,3.9,48,+91 9986210891,HSR,Quick Bites,Burgers,"Fast Food, Beverages",400,"[('Rated 4.0', 'RATED\n This little cafe is s...",[],Delivery,BTM
14,#refuel,"7, Ground Floor, RR Commercial Complex, Akshay...","['Cafes', 'Delivery', 'Dine-out']",https://www.zomato.com/bangalore/refuel-banner...,Yes,No,3.7,37,+91 8971227222,Bannerghatta Road,Cafe,"Thick Shakes, Sandwiches, Pasta, Mocktails","Cafe, Beverages",400,"[('Rated 3.0', 'RATED\n We ordered for Schezw...","['Kit Kat Thick Shake', 'Ferrero Rocher Thick ...",Cafes,Bannerghatta Road


In [9]:
all_ratings = []

for name,ratings in zip(newdata['name'],newdata['reviews_list']):
    ratings = eval(ratings)
    for score, doc in ratings:
        if score:
            score = score.strip("Rated").strip()
            doc = doc.strip('RATED').strip()
            score = float(score)
            all_ratings.append([name,score, doc])

In [10]:
import re

In [11]:
rating_df = pd.DataFrame(all_ratings, columns = ['name', 'rating', 'review'])
#rating_df['review'] = rating_df['review'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]', "", x))

In [12]:
rating_df.head()

,name,rating,review
0,#L-81 Cafe,4.0,This little cafe is set in a very beautiful lo...
1,#L-81 Cafe,4.0,"A cozy cafe near silk board, just came here be..."
2,#L-81 Cafe,5.0,Such a nice place to hang out. Its an open spa...
3,#L-81 Cafe,4.0,Love the ambiance. Perfect way to spend your S...
4,#L-81 Cafe,4.0,Such an amazing and beautiful place in the cal...


In [13]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

In [14]:
fig = px.histogram(rating_df, x="rating")
fig.update_traces(marker_color="turquoise",marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Ratings')
fig.show()

In [15]:
# converting to lower text
#rating_df['review'] = rating_df['review'].apply(lambda x: x.lower())

In [16]:
rating_df.head()

,name,rating,review
0,#L-81 Cafe,4.0,This little cafe is set in a very beautiful lo...
1,#L-81 Cafe,4.0,"A cozy cafe near silk board, just came here be..."
2,#L-81 Cafe,5.0,Such a nice place to hang out. Its an open spa...
3,#L-81 Cafe,4.0,Love the ambiance. Perfect way to spend your S...
4,#L-81 Cafe,4.0,Such an amazing and beautiful place in the cal...


In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vithyashagar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
tokenizer = RegexpTokenizer(r'\w+')

stopWords = set(stopwords.words("english"))

lemmatizer = WordNetLemmatizer()

In [19]:
rating_df.head()

,name,rating,review
0,#L-81 Cafe,4.0,This little cafe is set in a very beautiful lo...
1,#L-81 Cafe,4.0,"A cozy cafe near silk board, just came here be..."
2,#L-81 Cafe,5.0,Such a nice place to hang out. Its an open spa...
3,#L-81 Cafe,4.0,Love the ambiance. Perfect way to spend your S...
4,#L-81 Cafe,4.0,Such an amazing and beautiful place in the cal...


In [20]:
#Removing stopwords
def clean_sent(text):
    text = text.lower()

    #tokenize
    tokens = tokenizer.tokenize(text)
    new_tokens = [token for token in tokens if token not in stopWords]

    #lemmatizing
    stemmed_tokens = [lemmatizer.lemmatize(tokens) for tokens in new_tokens]

    
    return " ".join(stemmed_tokens)

In [21]:
rating_df["review"] = rating_df["review"].apply(lambda x: clean_sent(x))

In [22]:
rating_df.head()

,name,rating,review
0,#L-81 Cafe,4.0,little cafe set beautiful location ambiance go...
1,#L-81 Cafe,4.0,cozy cafe near silk board came boarding bus to...
2,#L-81 Cafe,5.0,nice place hang open space must visit place ni...
3,#L-81 Cafe,4.0,love ambiance perfect way spend sunday food re...
4,#L-81 Cafe,4.0,amazing beautiful place calm area hsr usually ...


In [23]:
rating_df['sentiment'] = rating_df['rating'].apply(lambda x: 1 if float(x)>3.0 else 0)
rating_df.head()

,name,rating,review,sentiment
0,#L-81 Cafe,4.0,little cafe set beautiful location ambiance go...,1
1,#L-81 Cafe,4.0,cozy cafe near silk board came boarding bus to...,1
2,#L-81 Cafe,5.0,nice place hang open space must visit place ni...,1
3,#L-81 Cafe,4.0,love ambiance perfect way spend sunday food re...,1
4,#L-81 Cafe,4.0,amazing beautiful place calm area hsr usually ...,1


In [24]:
# random split train and test data
index = rating_df.index
rating_df['random_number'] = np.random.randn(len(index))

In [25]:
train = rating_df[rating_df['random_number'] <= 0.8]
test = rating_df[rating_df['random_number'] > 0.8]

X_train = train["review"]
X_test = test["review"]
y_train = train['sentiment']
y_test = test['sentiment']

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range = (1,2))
XCV_train = cv.fit_transform(X_train).toarray()
XCV_test = cv.transform(X_test).toarray()
#XCV_train

In [27]:
XCV_train.shape

(33, 1029)

In [28]:
#print(cv.get_feature_names())

In [29]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [30]:
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
lr.fit(XCV_train,y_train)
y_pred = lr.predict(XCV_test)
confusion_matrix(y_pred,y_test)

array([[0, 0],
       [1, 8]], dtype=int64)

In [32]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.89      0.94         9

    accuracy                           0.89         9
   macro avg       0.50      0.44      0.47         9
weighted avg       1.00      0.89      0.94         9



C:\Users\Vithyashagar\anaconda3\envs\venev\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

C:\Users\Vithyashagar\anaconda3\envs\venev\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

C:\Users\Vithyashagar\anaconda3\envs\venev\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [33]:
test_file = pd.read_csv("New folder/ratings.csv")

In [34]:
test_file.head()

,rating,review
0,4.0,little cafe set beautiful location ambiance go...
1,4.0,cozy cafe near silk board came boarding bus to...
2,5.0,nice place hang open space must visit place ni...
3,4.0,love ambiance perfect way spend sunday food re...
4,4.0,amazing beautiful place calm area hsr usually ...


In [35]:
test_file = pd.DataFrame(test_file)
cleaned = test_file["review"].apply(lambda x: clean_sent(x))

predict_text = cv.transform(cleaned)

pred = lr.predict(predict_text)

In [36]:
cleaned

0     little cafe set beautiful location ambiance go...
1     cozy cafe near silk board came boarding bus to...
2     nice place hang open space must visit place ni...
3     love ambiance perfect way spend sunday food re...
4     amazing beautiful place calm area hsr usually ...
5     ordered schezwan paneer sandwich combo taste g...
6     nice compact place outdoor seating ordered veg...
7     ordered chicken subwich 49 offer suggestion pl...
8     tastiest sandwich theyve got milano sandwich m...
9     ive chef special mutton sub bread used low qua...
10    want best shawarmas town 1000 bc place go head...
11    frod closed hotel chicken supply hotel gave u ...
12    searching place would serve purpose ie quality...
13    paid 140 200 ml container biryani ppl fooling ...
14    small outlet actually stand name small chicken...
15    came take parcel dinner cook preferred finish ...
16    lunch friend 100 c restaurant food absolutely ...
17    focus chef kitchen staff procure premium q

In [37]:
test_file["prdeictions"] = pred.tolist()

In [38]:
test_file.to_csv("predicted.csv")